# Trabajo final data mining

En este trabajo brindaremos un chat bot de respuestas cortas para IM en español. Primero que nada definiremos una respuesta corta como cualquier turno hasta 3 palabras que ocurren al menos n veces. Iremos probando diferentes n a modo de prueba. El texto anterior a la respuesta corta será llamado contexto

## Extracción del corpus

Debido a que no hay un corpus bien definido para este problema utilizaremos el contexto del chat como texto de entrenamiento, por lo que la respuesta corta será la etiqueta.

### Procesamiento

In [2]:
import re
import csv
#import pandas as pd
from datetime import datetime, timedelta
from string import punctuation

#### Conociendo los datos "crudos"

In [3]:
with open('whatsapp/chat2.txt') as f:
    raw_data = f.readlines()
print(raw_data[:20])

['1/22/16, 18:47 - Lore Bracamonte: Hola Puppy. Te parece si nosotras nos juntamos un ratito antes de las 20 en el Olmos??\n', '1/22/16, 19:06 - Daniela Bosch: Dale!\n', '1/22/16, 19:06 - Daniela Bosch: Yo estoy cerca del centro\n', '1/22/16, 19:06 - Daniela Bosch: Así que en un rato salgo\n', '1/22/16, 19:15 - Lore Bracamonte: Bien, yo ya estoy por salir de mi casa\n', '1/22/16, 19:15 - Daniela Bosch: Ok\n', '1/22/16, 20:08 - Lore Bracamonte: Ya estoy en el Olmos Puppy\n', '1/23/16, 01:19 - Lore Bracamonte: Acabo de tomar el colectivo\n', '1/23/16, 01:20 - Daniela Bosch: Yo creo que el mío ya pasó\n', '1/23/16, 01:23 - Lore Bracamonte: Oh, espero que no\n', '1/23/16, 01:24 - Lore Bracamonte: Que este demorado nada más\n', '1/23/16, 01:25 - Daniela Bosch: No sw\n', '1/23/16, 01:28 - Lore Bracamonte: Que  vas a hacer si se paso??\n', '1/23/16, 01:33 - Daniela Bosch: Me tomo un taxi\n', '1/23/16, 01:36 - Daniela Bosch: Ya me tomé el 60\n', '1/23/16, 01:46 - Lore Bracamonte: Que bueno\n',

Sacaremos la primera línea que de nada sirve

`6/18/17, 16:43 - Los mensajes y llamadas en este chat ahora están protegidos con cifrado de extremo a extremo. Toca para más información.`

#### Tratando de preprocesar

Para ello spliteo cada línea por la primera ocurrencia de ": ". Ver bien si se puede hacer algo con las líneas que no tengan el mismo formato (por ahora la quitamos). También quitaremos las stops words y reduciremos ocurrencias de jaja a ja (por ahora hacerlo bien básico).

In [4]:
STOP_WORDS = punctuation

def clean_text(text):
    return re.sub('{1}{0}{1}'.format(STOP_WORDS, '[]'), '', text).lower()

def remove_ja(text):
    return re.sub('(ja)+', 'ja', text)

In [5]:
info_chat = []

for line in raw_data:
    if "-" not in line or ": " not in line:
        continue
    info, text = line.split(": ", 1)
    if '<Archivo omitido>' in text or '<Media omitted>' in text:
        continue
    date, owner = info.split(" - ", 1)
    date, hour = date.split(", ")
    hours, mins = hour.split(":")
    month, day, year = date.split("/")
    cleaning_text = remove_ja(clean_text(text.split('\n')[0]))
    info_chat.append([owner, datetime(int('20' + year), int(month), int(day), int(hours), int(mins)), cleaning_text])
info_chat

[['Lore Bracamonte',
  datetime.datetime(2016, 1, 22, 18, 47),
  'hola puppy te parece si nosotras nos juntamos un ratito antes de las 20 en el olmos'],
 ['Daniela Bosch', datetime.datetime(2016, 1, 22, 19, 6), 'dale'],
 ['Daniela Bosch',
  datetime.datetime(2016, 1, 22, 19, 6),
  'yo estoy cerca del centro'],
 ['Daniela Bosch',
  datetime.datetime(2016, 1, 22, 19, 6),
  'así que en un rato salgo'],
 ['Lore Bracamonte',
  datetime.datetime(2016, 1, 22, 19, 15),
  'bien yo ya estoy por salir de mi casa'],
 ['Daniela Bosch', datetime.datetime(2016, 1, 22, 19, 15), 'ok'],
 ['Lore Bracamonte',
  datetime.datetime(2016, 1, 22, 20, 8),
  'ya estoy en el olmos puppy'],
 ['Lore Bracamonte',
  datetime.datetime(2016, 1, 23, 1, 19),
  'acabo de tomar el colectivo'],
 ['Daniela Bosch',
  datetime.datetime(2016, 1, 23, 1, 20),
  'yo creo que el mío ya pasó'],
 ['Lore Bracamonte',
  datetime.datetime(2016, 1, 23, 1, 23),
  'oh espero que no'],
 ['Lore Bracamonte',
  datetime.datetime(2016, 1, 23, 1

Ahora que tenemos los datos un poco más limpios utilizaremos panda para sacar un poco de conclusiones.

In [5]:
df = pd.DataFrame(info_chat, columns=['owner', 'date', 'text'])
df

,owner,date,text
0,Lore Bracamonte,2016-01-22 18:47:00,hola puppy te parece si nosotras nos juntamos ...
1,Daniela Bosch,2016-01-22 19:06:00,dale
2,Daniela Bosch,2016-01-22 19:06:00,yo estoy cerca del centro
3,Daniela Bosch,2016-01-22 19:06:00,así que en un rato salgo
4,Lore Bracamonte,2016-01-22 19:15:00,bien yo ya estoy por salir de mi casa
5,Daniela Bosch,2016-01-22 19:15:00,ok
6,Lore Bracamonte,2016-01-22 20:08:00,ya estoy en el olmos puppy
7,Lore Bracamonte,2016-01-23 01:19:00,acabo de tomar el colectivo
8,Daniela Bosch,2016-01-23 01:20:00,yo creo que el mío ya pasó
9,Lore Bracamonte,2016-01-23 01:23:00,oh espero que no


Ahora agruparemos los contextos con sus respectivas respuestas. Guardaremos el corpus en formato CSV.

In [6]:
MIN = 60
HOUR = 60*MIN
SAME_CONTEXT = timedelta(seconds=24*HOUR)
SAME_TEXT = timedelta(seconds=30*MIN)

SHORT_ANSWER = 2
def is_short(answer):
    """
    is short answer?
    """
    return len(answer.split(' ')) <= SHORT_ANSWER

def is_same_context(hour1, hour2):
    """
    hour1 must be larger-equal than hour2
    """
    return hour1 - hour2 <= SAME_CONTEXT

def is_same_text(hour1, hour2):
    return hour1 - hour2 <= SAME_TEXT

def is_same_owner(own1, own2):
    """
    return whenever own1 is the same as own2
    """
    return own1==own2

In [8]:
corpus_data = []

with open('corpus.csv', 'w', newline='') as csvfile:
    fieldnames = ['context', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, dialect='unix')
    writer.writeheader()

    last_owner = ''
    last_time = datetime(2000, 1, 1)
    row = {'context':'', 'label':''}
    for i, (owner, time, text) in enumerate(info_chat):
        if is_short(text):                
            if not is_same_owner(last_owner, owner):
                # si no es la misma persona que escribe quiere decir que hubo una respuesta (en este caso corta).
                # el label será la respuesta en sí. Empezará un contexto nuevo de la otra persona.
                row['label'] = text
                writer.writerow(row)
                row['context'] = text
            else:
                # si la misma persona sigue escribiendo
                if is_same_text(time, last_time):
                    # si es parte de la misma conversación, agregamos al contexto
                    row['context'] += ' ' + text
                else:
                    # si "cambia la conversación", podríamos decir que la otra persona no contesto y por ello
                    # etiquetariamos como nonDef
                    row['label'] = 'nonDef'
                    writer.writerow(row)
                    row['context'] = text
        else:
            # si no es una respuesta corta
            if is_same_text(time, last_time):
                row['context'] += ' ' + text
            else:
                row['label'] = 'nonDef'
                writer.writerow(row)
                row['context'] = text

        last_owner = owner
        last_time = time

In [14]:
# Contexto con ventana de 10 conversaciones
FRAME = 10

with open('corpus2.csv', 'w', newline='') as csvfile:
    fieldnames = ['context', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, dialect='unix')
    writer.writeheader()
    
    # No uso el datetime
    # Tampoco tengo en cuenta el usuario que escribe
    row = {'context':[], 'label':''}
    for owner, time, text in info_chat:
        if is_short(text):
            row['label'] = text
            writer.writerow(row)

        # Descartamos la primer conversación en la lista 
        # y agregamos la última 
        if len(row['context']) == FRAME:
            row['context'] = row['context'][1:]
            row['context'].append(text)
        else:
            row['context'].append(text)

In [21]:
from collections import defaultdict

short_answers = {}
short_answers = defaultdict(lambda: 0, short_answers)
with open('corpus2.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        short_answers[row['label']] += 1
        
sorted(short_answers.items(), key=lambda x: x[1], reverse=True)

[('ja', 56),
 ('oki', 50),
 ('puppy', 45),
 ('😁', 36),
 ('lore', 32),
 ('dale', 31),
 ('ok', 24),
 ('genial', 21),
 ('hola puppy', 13),
 ('oka', 11),
 ('🙂', 11),
 ('hola lore', 10),
 ('sipi', 9),
 ('sip', 9),
 ('😊', 7),
 ('😋', 6),
 ('copado', 5),
 ('ja dale', 5),
 ('bueno', 5),
 ('llegué', 5),
 ('siii', 4),
 ('😕', 4),
 ('', 4),
 ('si dale', 4),
 ('sii', 4),
 ('gracias puppy', 4),
 ('auch', 4),
 ('vamos', 3),
 ('a ver', 3),
 ('sí', 3),
 ('a verr', 3),
 ('si', 3),
 ('😱', 3),
 ('como va', 3),
 ('ya llegué', 3),
 ('claro', 3),
 ('ahí voy', 3),
 ('ah dale', 3),
 ('ja ok', 3),
 ('lore 👋🏼', 3),
 ('🤔', 3),
 ('😙', 3),
 ('estoy yendo', 2),
 ('viernes', 2),
 ('sep', 2),
 ('estoy llegando', 2),
 ('jeje', 2),
 ('ahh ja', 2),
 ('uhhh', 2),
 ('eso', 2),
 ('ñoña', 2),
 ('te pinta', 2),
 ('mira lore', 2),
 ('buenisimo', 2),
 ('uh', 2),
 ('⁉', 2),
 ('que moco', 2),
 ('ah oki', 2),
 ('okii', 2),
 ('gracias', 2),
 ('😅', 2),
 ('quedó genial', 2),
 ('😄', 2),
 ('ni idea', 2),
 ('cómo va', 2),
 ('wii', 2),
 (